# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Background-Description" data-toc-modified-id="Background-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Background Description</a></div><div class="lev1 toc-item"><a href="#Configuration-Settings" data-toc-modified-id="Configuration-Settings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configuration Settings</a></div><div class="lev2 toc-item"><a href="#Check-the-settings" data-toc-modified-id="Check-the-settings-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Check the settings</a></div><div class="lev2 toc-item"><a href="#Critical-patameters" data-toc-modified-id="Critical-patameters-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Critical patameters</a></div><div class="lev2 toc-item"><a href="#Modify-the-settings" data-toc-modified-id="Modify-the-settings-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Modify the settings</a></div><div class="lev3 toc-item"><a href="#ALTER-DATABASE--vs--ALTER-SYSTEM" data-toc-modified-id="ALTER-DATABASE--vs--ALTER-SYSTEM-231"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span><code>ALTER DATABASE</code>  vs  <code>ALTER SYSTEM</code></a></div><div class="lev4 toc-item"><a href="#ALTER-SYSTEM-Synopsis" data-toc-modified-id="ALTER-SYSTEM-Synopsis-2311"><span class="toc-item-num">2.3.1.1&nbsp;&nbsp;</span>ALTER SYSTEM Synopsis</a></div><div class="lev4 toc-item"><a href="#ALTER-DATABASE-Synopsis" data-toc-modified-id="ALTER-DATABASE-Synopsis-2312"><span class="toc-item-num">2.3.1.2&nbsp;&nbsp;</span>ALTER DATABASE Synopsis</a></div><div class="lev3 toc-item"><a href="#Change-the-number" data-toc-modified-id="Change-the-number-232"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Change the number</a></div><div class="lev3 toc-item"><a href="#Reload-the-settings" data-toc-modified-id="Reload-the-settings-233"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Reload the settings</a></div><div class="lev3 toc-item"><a href="#Restart-the-server" data-toc-modified-id="Restart-the-server-234"><span class="toc-item-num">2.3.4&nbsp;&nbsp;</span>Restart the server</a></div><div class="lev2 toc-item"><a href="#Summary" data-toc-modified-id="Summary-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Summary</a></div><div class="lev1 toc-item"><a href="#Update-Statistics-of-database-(!!!)" data-toc-modified-id="Update-Statistics-of-database-(!!!)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Update Statistics of database (!!!)</a></div><div class="lev2 toc-item"><a href="#VACUUM" data-toc-modified-id="VACUUM-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>VACUUM</a></div><div class="lev2 toc-item"><a href="#ANALYZE-(!!!)" data-toc-modified-id="ANALYZE-(!!!)-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>ANALYZE (!!!)</a></div><div class="lev3 toc-item"><a href="#Synopsis" data-toc-modified-id="Synopsis-321"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Synopsis</a></div><div class="lev3 toc-item"><a href="#Description" data-toc-modified-id="Description-322"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Description</a></div><div class="lev1 toc-item"><a href="#Write-Better-Queries" data-toc-modified-id="Write-Better-Queries-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Write Better Queries</a></div><div class="lev2 toc-item"><a href="#Some-advice" data-toc-modified-id="Some-advice-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Some advice</a></div><div class="lev2 toc-item"><a href="#let-postgresql-test-the-query-before-execution" data-toc-modified-id="let-postgresql-test-the-query-before-execution-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>let postgresql test the query before execution</a></div><div class="lev1 toc-item"><a href="#References" data-toc-modified-id="References-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>References</a></div><div class="lev1 toc-item"><a href="#Appendix" data-toc-modified-id="Appendix-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Appendix</a></div><div class="lev2 toc-item"><a href="#Check-users-and-attributes" data-toc-modified-id="Check-users-and-attributes-61"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Check users and attributes</a></div>

# Background Description

Our ETL system execution in Python and PostgreSQL is somehow too slow.  
So we'd like to increase the effectiveness of execution by making good use of PostgreSQL tools.  

# Configuration Settings

## Check the settings

Enter the postgresql interactive terminal by:
```
psql -U <user_name> -d <database_name> -h <127.0.0.1> -p <5432>
```

And then run:
```
SELECT name, context, unit, 
    setting, boot_val, reset_val
FROM pg_settings
WHERE name IN (
	'listen_addresses',
	'max_connections',
	'shared_buffers',
	'effective_cache_size',
	'work_mem',
	'maintenance_work_mem'
)
ORDER BY context, name;
```

The output looks like below:
```
         name         |  context   | unit |  setting  | boot_val  | reset_val 
----------------------+------------+------+-----------+-----------+-----------
 listen_addresses     | postmaster |      | localhost | localhost | localhost
 max_connections      | postmaster |      | 100       | 100       | 100
 shared_buffers       | postmaster | 8kB  | 16384     | 1024      | 16384
 effective_cache_size | user       | 8kB  | 524288    | 524288    | 524288
 maintenance_work_mem | user       | kB   | 65536     | 65536     | 65536
 work_mem             | user       | kB   | 4096      | 4096      | 4096
(6 rows)
```

* Pay attention to the unit.

Or just use:
```
SHOW work_mem;
```
Then the output will be in readable type:
```
 work_mem 
----------
 4MB
(1 row)
```

Now the settings of postgresql on the server is: 
```
         name         |  context   | unit |  setting  | boot_val  | reset_val 
----------------------+------------+------+-----------+-----------+-----------
 listen_addresses     | postmaster |      | localhost | localhost | localhost
 max_connections      | postmaster |      | 100       | 100       | 100
 shared_buffers       | postmaster | 8kB  | 524288    | 1024      | 524288
 effective_cache_size | user       | 8kB  | 1048576   | 524288    | 1048576
 maintenance_work_mem | user       | kB   | 1048576   | 65536     | 1048576
 work_mem             | user       | kB   | 8388608   | 4096      | 8388608
(6 rows)
```

## Critical patameters

__Pay special attention to the following network settings in postgresql.conf; changing their values requires a service restart.__


* __listen_addresses__

    Informs PostgreSQL which IP addresses to listen on. This usually defaults to localhost or local, but many people change it to *, meaning all available IP addresses.


* __port__

    Defaults to 5432. If you happen to be on Red Hat or CentOS, make changes to the PGPORT value /etc/sysconfig/pgsql/your_service_name_here to change the listening port.


* __max_connections__

    The maximum number of concurrent connections allowed.

__In our experience, we found the following three settings to affect performance across the board and might be worthy of experimentation for your particular setup:__

* __shared_buffers__

    Defines the amount of memory shared among all connections to store recently accessed pages. 
    
    This setting profoundly affects the speed of your queries. You want this setting to be fairly high, probably as much as 25% of your onboard memory. However, you’ll generally see diminishing returns after more than 8 GB. Changes require a restart.


* __effective_cache_size__
    
    An estimate of how much memory you expect to be available in the OS and PostgreSQL buffer caches. This setting has no effect on actual allocation, but query planner figures in this setting to guess whether intermediate steps and query output would fit in RAM. If you set this much lower than available RAM, the planner may forgo using indexes. 

    With a dedicated server, setting effective_cache_size to half or more of your onboard memory would be a good start. Changes require at least a reload.


* __maintenance_work_mem__

    The total memory allocated for housekeeping activities such as vacuuming (pruning records marked for delete). 
    
    You shouldn’t set it higher than about 1 GB. Reload after changes.
    

* __work_mem__

    Controls the maximum amount of memory allocated for operations such as sorting, hash join, and table scans. The optimal setting depends on how you’re using the database, how much memory you have to spare, and whether your server is dedicated to PostgreSQL or not. If you have many users running simple queries, you want this setting to be relatively low. How high you set this also depends on how much RAM you have to begin with. 
    
    Changes require at least a reload.


## Modify the settings

### `ALTER DATABASE`  vs  `ALTER SYSTEM`

|| ALTER DATABASE | ALTER SYSTEM |
|--|--|--|
| Definition | ALTER DATABASE changes the attributes of a database. | ALTER SYSTEM is used for changing server configuration parameters across the entire database cluster. |
| Who can do the changes | database owner | superuser |
| Where to take effect | database | system |
| How to take effect | after reload | after create new database |
| Change a configuration parameter | Yes | Yes |
| Change owner/name/... of database | Yes | No |

#### ALTER SYSTEM Synopsis
```
ALTER SYSTEM SET configuration_parameter { TO | = } { value | 'value' | DEFAULT }

ALTER SYSTEM RESET configuration_parameter
ALTER SYSTEM RESET ALL
```

#### ALTER DATABASE Synopsis
```
ALTER DATABASE name [ [ WITH ] option [ ... ] ]

where option can be:

    CONNECTION LIMIT connlimit

ALTER DATABASE name RENAME TO new_name

ALTER DATABASE name OWNER TO new_owner

ALTER DATABASE name SET TABLESPACE new_tablespace

ALTER DATABASE name SET configuration_parameter { TO | = } { value | DEFAULT }
ALTER DATABASE name SET configuration_parameter FROM CURRENT
ALTER DATABASE name RESET configuration_parameter
ALTER DATABASE name RESET ALL
```

### Change the number

The parameters which require "a reload" are:

`shared_buffers`, `effective_cache_size`, `work_mem`, `maintenance_work_mem`

__Set__ the parameter config by sentence in PostgreSQL: 
```
ALTER DATABASE set work_mem = 8192;
```
in the unit shown above in the output table, or specify the unit when setting: 
```
ALTER DATABASE set work_mem = '8MB';
```

### Reload the settings

Then __reload__ config info, using one of the methods: 
* in PostgreSQL as a superuser: ```SELECT pg_reload_conf();``` 
* in Ubuntu terminal : ```service postgresql-9.5 reload``` （更推荐这种方式）

### Restart the server

Enter the Ubuntu terminal and type:
```
sudo service postgresql restart
```

## Summary

| Parameter | Meaning | Default Value |Suggested Value | Need Reload or Restart | ALTER SYSTEM or DATABASE |
|--|--|--|--|--|
| listen_addresses     | Informs PostgreSQL which IP addresses to listen on. | localhost | * | Restart | SYSTEM |
| max_connections      | The maximum number of concurrent connections allowed. | 100 |  | Restart | SYSTEM |
| shared_buffers       | The amount of memory shared among all connections to store recently accessed pages. | 128MB | 0.25 * onboard-memory | Restart | SYSTEM |
| effective_cache_size | An estimate of how much memory you expect to be available in the OS and PostgreSQL buffer caches. | 4GB | >= 0.5 * onboard-memory | Reload | DATABASE |
| maintenance_work_mem | The total memory allocated for housekeeping activities such as vacuuming (pruning records marked for delete).  | 64MB | <= 1GB | Reload | DATABASE |
| work_mem             | the maximum amount of memory allocated for operations such as sorting, hash join, and table scans. | 4MB |  | Reload | DATABASE |

# Update Statistics of database (!!!)

## VACUUM

After a large data load or a major deletion, you should manually update the stats by executing VACUUM ANALYZE.  
VACUUM permanently removes deleted rows from tables; ANALYZE updates the stats.

## ANALYZE (!!!)

### Synopsis
```
ANALYZE [ VERBOSE ] [ table [ ( column [, ...] ) ] ]
```
### Description

ANALYZE collects statistics about the contents of tables in the database, and stores the results in the pg_statistic system catalog. Subsequently, the query planner uses these statistics to help determine the most efficient execution plans for queries.

With no parameter, ANALYZE examines every table in the current database. With a parameter, ANALYZE examines only that table. It is further possible to give a list of column names, in which case only the statistics for those columns are collected.

Accurate statistics will help the planner to choose the most appropriate query plan, and thereby improve the speed of query processing. A common strategy is to run VACUUM and ANALYZE once a day during a low-usage time of day.

ANALYZE requires only a read lock on the target table, so it can run in parallel with other activity on the table.


# Write Better Queries 

## Some advice
* Overusing subqueries in select
* Avoid `SELECT *`
* Make Good Use of CASE

## let postgresql test the query before execution
Cost too much time in practice. 
```
EXPLAIN (ANALYZE, VERBOSE, BUFFERS) your_query_here;
```


# References

* postgresql_server_configure_settings
    * [PostgreSQL Up & Running (safaribooksonline)](https://www.safaribooksonline.com/library/view/postgresql-up-and/9781449373184/ch02.html)
    * [PostgreSQL Documentation (official documentation)](https://www.postgresql.org/docs/9.5/static/index.html)
    * [Tuning Your PostgreSQL Server (postgresql wiki)](https://wiki.postgresql.org/wiki/Tuning_Your_PostgreSQL_Server)

# Appendix

## Check users and attributes

```
SELECT * FROM pg_user;
```

```
   usename   | usesysid | usecreatedb | usesuper | userepl | usebypassrls |  passwd  | valuntil | useconfig 
-------------+----------+-------------+----------+---------+--------------+----------+----------+-----------
 datascience |    16384 | f           | t        | f       | f            | ******** |          | 
 postgres    |       10 | t           | t        | t       | f            | ******** |          | 
(2 rows)
```

A database user may have a number of attributes that define its privileges and interact with the client authentication system.

superuser

    A database superuser bypasses all permission checks. Also, only a superuser can create new users. To create a database superuser, use CREATE USER name CREATEUSER.
    
database creation

    A user must be explicitly given permission to create databases (except for superusers, since those bypass all permission checks). To create such a user, use CREATE USER name CREATEDB.
    
password

    A password is only significant if the client authentication method requires the user to supply a password when connecting to the database. The password, md5, and crypt authentication methods make use of passwords. Database passwords are separate from operating system passwords. Specify a password upon user creation with CREATE USER name PASSWORD 'string'.

